In [15]:
# As we use our own external modules, we need the folder src to be in the PYTHONPATH env variable.
# However we do not expect the reader to add that folder to the env variable,
# therefore we manually load it temporarily in each notebook.
import os
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [16]:
import pandas as pd
from timeit import default_timer as timer
from utils.config import PATH_SCENARIOS, N_REDUCED_SCNEARIOS, PATH_SCENARIOS_REDUCED
from sklearn_extra.cluster import KMedoids

In [17]:
scenarios = pd.read_pickle(PATH_SCENARIOS)

In [18]:
scenarios = scenarios.unstack(level=['start_hex_ids', 'end_hex_ids', 'time', 'vehicle_types'])

In [19]:
X = scenarios.values

In [20]:
kmedoids = KMedoids(n_clusters=N_REDUCED_SCNEARIOS, random_state=0).fit(X)

In [21]:
# note that because the scenarios dataframe was sorted the indeces of the medoids are also the 
# scenario ids
selected_scenario_ids = kmedoids.medoid_indices_

In [22]:
selected_scenarios = scenarios.loc[selected_scenario_ids]

In [23]:
selected_scenarios = selected_scenarios.stack(['start_hex_ids', 'end_hex_ids', 'time', 'vehicle_types'])

In [25]:
scenario_id_list = list(selected_scenarios.index.get_level_values('scenarios').unique())
scenario_reset_map = {id:i for i, id in enumerate(scenario_id_list)}

In [27]:
selected_scenarios = selected_scenarios.rename(index=scenario_reset_map)

In [28]:
os.makedirs(os.path.dirname(PATH_SCENARIOS_REDUCED), exist_ok=True)
selected_scenarios.to_pickle(PATH_SCENARIOS_REDUCED)